Based on [climate-assessment example notebook](https://github.com/iiasa/climate-assessment/blob/main/notebooks/run-example-fair.ipynb).

In [1]:
import jupyter_black
jupyter_black.load()

In [2]:
import logging

# Increase the level to reduce the number of log messages
LOGGING_LEVEL = logging.INFO

LOGGER = logging.getLogger("pipeline")
LOGGER.setLevel(LOGGING_LEVEL)
# have to set root logger too to get messages to come through
logging.getLogger().setLevel(LOGGING_LEVEL)

logFormatter = logging.Formatter(
    "%(asctime)s %(name)s %(threadName)s - %(levelname)s:  %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)
stdoutHandler = logging.StreamHandler()
stdoutHandler.setFormatter(logFormatter)

logging.getLogger().addHandler(stdoutHandler)

In [3]:
import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)

import pooch

import matplotlib.pyplot as plt
import pandas as pd
import pyam

from climate_assessment.cli import run_workflow

2023-09-04 11:53:30 pint.util MainThread - WARNING:  Redefining 'kt' (<class 'pint.definitions.UnitDefinition'>)


<IPython.core.display.Javascript object>

2023-09-04 11:53:31 pint.util MainThread - WARNING:  Redefining 'C' (<class 'pint.definitions.UnitDefinition'>)
2023-09-04 11:53:31 pint.util MainThread - WARNING:  Redefining 'N' (<class 'pint.definitions.UnitDefinition'>)
2023-09-04 11:53:31 pint.util MainThread - WARNING:  Redefining 'NOX' (<class 'pint.definitions.UnitDefinition'>)
2023-09-04 11:53:31 pint.util MainThread - WARNING:  Redefining 'gNOX' (<class 'pint.definitions.UnitDefinition'>)
2023-09-04 11:53:31 pint.util MainThread - WARNING:  Redefining 'tNOX' (<class 'pint.definitions.UnitDefinition'>)
2023-09-04 11:53:31 pint.util MainThread - WARNING:  Redefining 'S' (<class 'pint.definitions.UnitDefinition'>)
2023-09-04 11:53:31 pint.util MainThread - WARNING:  Redefining 'a' (<class 'pint.definitions.UnitDefinition'>)
2023-09-04 11:53:31 pint.util MainThread - WARNING:  Redefining 'yr' (<class 'pint.definitions.UnitDefinition'>)
2023-09-04 11:53:31 pint.util MainThread - WARNING:  Redefining 'h' (<class 'pint.definitions.U

### Download FaIR configuration

In [4]:
fair_slim_filename = "fair-1.6.2-wg3-params-slim.json"
fair_common_filename = "fair-1.6.2-wg3-params-common.json"

In [5]:
fair_slim_url = (
    "https://zenodo.org/record/6601980/files/fair-1.6.2-wg3-params-slim.json?download=1"
)
fair_slim_hash = "c071ca619c0ae37a6abdeb79c0cece7b"

pooch.retrieve(
    url=fair_slim_url,
    known_hash=f"md5:{fair_slim_hash}",
    path=".",
    fname=fair_slim_filename,
)

'/home/robert/Work/polyclimate/polyclimate1-global-projections/notebooks/proof-of-concept/fair-1.6.2-wg3-params-slim.json'

In [6]:
fair_common_url = "https://zenodo.org/record/6601980/files/fair-1.6.2-wg3-params-common.json?download=1"
fair_common_hash = "42ccaffcd3dea88edfca77da0cd5789b"

pooch.retrieve(
    url=fair_common_url,
    known_hash=f"md5:{fair_common_hash}",
    path=".",
    fname=fair_common_filename,
)

'/home/robert/Work/polyclimate/polyclimate1-global-projections/notebooks/proof-of-concept/fair-1.6.2-wg3-params-common.json'

### General input arguments and options to the climate assessment workflow

In [7]:
model = "fair"
model_version = "1.6.2"
fair_extra_config = fair_common_filename
probabilistic_file = fair_slim_filename

# Use fewer (e.g. 10) if you just want to do a test run but note that this breaks
# the stats of the probabilistic ensemble
num_cfgs = 2237
# Set to True if you're not using the full FaIR ensemble
test_run = False
# How many scenarios do you want to run in one go?
scenario_batch_size = 20

outdir = "."

### Input emissions pathway file

By default, we use 2 of the AR6 IPs. You could use a different file (formatted the same way) if you wish.

In [8]:
input_emissions_file = "polyclimate.csv"

### Choose infiller database file

We run using the infiller database that was used in CMIP6. As a result of the licensing of the scenario data, this file has to be downloaded by hand (see documentation under "Installation", section "Infiller database"). Make sure that the variable `infilling_database_file` points to where you saved this file.

In [9]:
infilling_database_file = (
    "1652361598937-ar6_emissions_vetted_infillerdatabase_10.5281-zenodo.6390768.csv"
)

## Run the climate assessment workflow

*N.B. the log with information and some warnings will be quite long - but that is nothing to worry about!*

In [10]:
run_workflow(
    input_emissions_file,
    outdir,
    model=model,
    model_version=model_version,
    probabilistic_file=probabilistic_file,
    fair_extra_config=fair_extra_config,
    num_cfgs=num_cfgs,
    infilling_database=infilling_database_file,
    scenario_batch_size=scenario_batch_size,
    harmonize=True,
)

INFO:climate_assessment.cli:Outputs will be saved in: .
2023-09-04 11:53:32 climate_assessment.cli MainThread - INFO:  Outputs will be saved in: .
INFO:climate_assessment.cli:Outputs will be saved with the ID: polyclimate
2023-09-04 11:53:32 climate_assessment.cli MainThread - INFO:  Outputs will be saved with the ID: polyclimate
INFO:climate_assessment.cli:Loading polyclimate.csv
2023-09-04 11:53:32 climate_assessment.cli MainThread - INFO:  Loading polyclimate.csv
INFO:pyam.core:Reading file polyclimate.csv
2023-09-04 11:53:32 pyam.core MainThread - INFO:  Reading file polyclimate.csv
INFO:climate_assessment.cli:Converting to basic columns i.e. removing any extra columns
2023-09-04 11:53:32 climate_assessment.cli MainThread - INFO:  Converting to basic columns i.e. removing any extra columns
INFO:climate_assessment.cli:Performing input data checks
2023-09-04 11:53:32 climate_assessment.cli MainThread - INFO:  Performing input data checks
INFO:climate_assessment.checks:CHECK: if no no

### Load results

In [11]:
output = pyam.IamDataFrame("polyclimate_alloutput.xlsx")
output

<class 'pyam.core.IamDataFrame'>
Index:
 * model    : Polyclimate (1)
 * scenario : Minus-5, Persistence, Plus-1 (3)
Timeseries data coordinates:
   region   : World (1)
   variable : ... (622)
   unit     : K, Mt BC/yr, Mt CH4/yr, Mt CO/yr, Mt CO2-equiv/yr, ... ppm (57)
   year     : 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, ... 2100 (106)
Meta indicators:
   exclude (bool) False (1)
   Exceedance Probability 1.5C (FaIRv1.6.2) (float64) 0.5033527045149754, ... 1.0 (3)
   Exceedance Probability 2.0C (FaIRv1.6.2) (float64) 0.0447027268663388, ... 1.0 (3)
   Exceedance Probability 2.5C (FaIRv1.6.2) (float64) 0.0022351363433169, ... (3)
   Exceedance Probability 3.0C (FaIRv1.6.2) (float64) 0.0, ... 0.9275815824765312 (3)
   ...

In [12]:
output

<class 'pyam.core.IamDataFrame'>
Index:
 * model    : Polyclimate (1)
 * scenario : Minus-5, Persistence, Plus-1 (3)
Timeseries data coordinates:
   region   : World (1)
   variable : ... (622)
   unit     : K, Mt BC/yr, Mt CH4/yr, Mt CO/yr, Mt CO2-equiv/yr, ... ppm (57)
   year     : 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, ... 2100 (106)
Meta indicators:
   exclude (bool) False (1)
   Exceedance Probability 1.5C (FaIRv1.6.2) (float64) 0.5033527045149754, ... 1.0 (3)
   Exceedance Probability 2.0C (FaIRv1.6.2) (float64) 0.0447027268663388, ... 1.0 (3)
   Exceedance Probability 2.5C (FaIRv1.6.2) (float64) 0.0022351363433169, ... (3)
   Exceedance Probability 3.0C (FaIRv1.6.2) (float64) 0.0, ... 0.9275815824765312 (3)
   ...

In [13]:
output.variable

['AR6 climate diagnostics|Atmospheric Concentrations|CH4|FaIRv1.6.2|10.0th Percentile',
 'AR6 climate diagnostics|Atmospheric Concentrations|CH4|FaIRv1.6.2|16.7th Percentile',
 'AR6 climate diagnostics|Atmospheric Concentrations|CH4|FaIRv1.6.2|17.0th Percentile',
 'AR6 climate diagnostics|Atmospheric Concentrations|CH4|FaIRv1.6.2|25.0th Percentile',
 'AR6 climate diagnostics|Atmospheric Concentrations|CH4|FaIRv1.6.2|33.0th Percentile',
 'AR6 climate diagnostics|Atmospheric Concentrations|CH4|FaIRv1.6.2|5.0th Percentile',
 'AR6 climate diagnostics|Atmospheric Concentrations|CH4|FaIRv1.6.2|50.0th Percentile',
 'AR6 climate diagnostics|Atmospheric Concentrations|CH4|FaIRv1.6.2|66.0th Percentile',
 'AR6 climate diagnostics|Atmospheric Concentrations|CH4|FaIRv1.6.2|67.0th Percentile',
 'AR6 climate diagnostics|Atmospheric Concentrations|CH4|FaIRv1.6.2|75.0th Percentile',
 'AR6 climate diagnostics|Atmospheric Concentrations|CH4|FaIRv1.6.2|83.0th Percentile',
 'AR6 climate diagnostics|Atmosph